el mejor codigo hasta ahora

In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import optuna
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import optuna
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, SVMSMOTE, RandomOverSampler

data = pd.read_csv('df_entrenamiento.csv')


# Printing all column names
print("Column Names:")
print(data.columns.tolist()) 



columns_to_drop = ['Unnamed: 0.1', 'Unnamed: 0']
data.drop(columns=columns_to_drop, inplace=True)

#delete outliers
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)

# Calculate the IQR (Interquartile Range)
IQR = Q3 - Q1

# Define a threshold to detect outliers (e.g., 1.5 times IQR)
threshold = 1.5

# Filter out rows with outliers based on the threshold
data_no_outliers = data[~((data < (Q1 - threshold * IQR)) | (data > (Q3 + threshold * IQR))).any(axis=1)]

#borramos de nuevo para night activitu 
Q1 = data['nightactivity_2_to_5'].quantile(0.25)
Q3 = data['nightactivity_2_to_5'].quantile(0.75)

# Calculate the IQR (Interquartile Range) for the 'nightactivity_2_to_5' column
IQR = Q3 - Q1

# Define a threshold to detect outliers (e.g., 1.5 times IQR)
threshold = 1.5

# Filter out rows with outliers in the 'nightactivity_2_to_5' column based on the threshold
data_no_outliers = data[~((data['nightactivity_2_to_5'] < (Q1 - threshold * IQR)) | (data['nightactivity_2_to_5'] > (Q3 + threshold * IQR)))]


correlations = data.corr()['label']

# Filter columns based on correlation threshold
columns_to_keep = correlations[(correlations >= 0.1) | (correlations <= -0.1)].index.tolist()

# Create a new DataFrame with selected columns
data = data[columns_to_keep]

data


Column Names:
['Unnamed: 0.1', 'Unnamed: 0', 'grade', 'seq_4_avg', 'quiz_avg', 'seq_0_avg', 'seq_1_avg', 'seq_2_avg', 'seq_3_avg', 'duracion_sesion_avg', 'duracion_sesion_std', 'duracion_EOL', 'num_sesiones_agosto', 'num_sesiones_septiembre', 'num_sesiones_octubre', 'page_close_mean', 'page_close_std', 'problem_graded_mean', 'problem_graded_std', 'problem_check_mean', 'problem_check_std', 'problem_show_mean', 'problem_show_std', 'load_video_mean', 'load_video_std', 'play_video_mean', 'play_video_std', 'pause_video_mean', 'pause_video_std', 'speed_change_video_mean', 'speed_change_video_std', 'num_eventos_seq_0', 'num_eventos_seq_1', 'num_eventos_seq_2', 'num_eventos_seq_3', 'num_eventos_seq_4', 'avg_time_between_sessions', 'std_time_between_sessions', 'nightactivity_2_to_5', 'nightactivity_0_to_2', 'sesionesde_2_5', 'sesionesde_0_2', 'label']


,grade,quiz_avg,seq_0_avg,seq_1_avg,seq_2_avg,num_sesiones_agosto,problem_show_mean,problem_show_std,num_eventos_seq_2,num_eventos_seq_3,avg_time_between_sessions,label
0,1.000000,0.965517,1.000000,1.000000,1.000000,5.000000,0.000000,0.000000,128.000000,16.000000,7.111111,1
1,0.990000,0.956897,1.000000,0.977273,1.000000,7.000000,0.142857,0.534522,90.000000,16.000000,4.615385,1
2,1.000000,0.965517,1.000000,1.000000,1.000000,13.000000,0.000000,0.000000,151.000000,47.000000,2.950000,1
3,0.880000,0.850690,0.625000,0.909091,0.907778,9.000000,0.400000,0.632456,201.000000,33.000000,4.142857,1
4,1.000000,0.965517,1.000000,1.000000,1.000000,11.000000,0.000000,0.000000,80.000000,19.000000,3.411765,0
...,...,...,...,...,...,...,...,...,...,...,...,...
603,0.926868,0.856774,1.000000,0.742882,0.779122,6.674731,0.276098,0.525444,158.818561,10.397845,4.506014,0
604,0.926484,0.897527,0.866209,0.942383,0.910806,7.140659,0.322184,0.668790,155.428576,17.183517,4.455478,0
605,0.957893,0.927401,0.947313,0.846920,1.000000,5.789254,0.558957,1.552358,193.937910,16.789254,3.661868,0
606,0.869460,0.836817,0.717063,0.831517,0.833140,9.113751,0.108045,0.316410,265.604752,30.586753,3.184089,0


In [28]:


X = data.drop('label', axis=1)
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42, shuffle=True)

# Normalizing data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def objective(trial):
    # KNN parameters
    knn_n_neighbors = trial.suggest_int('knn_n_neighbors', 1, 50)
    knn_weights = trial.suggest_categorical('knn_weights', ['uniform', 'distance'])
    
    # Decision Tree parameters
    dt_max_depth = trial.suggest_int('dt_max_depth', 1, 10)
    dt_criterion = trial.suggest_categorical('dt_criterion', ['gini', 'entropy'])
    
    sampling_type = trial.suggest_categorical("sampling_type", ["RandomOverSampler", "SMOTE", "BorderlineSMOTE", "SVMSMOTE"])
    
    # Seleccionar el tipo de oversampling/SMOTE según la sugerencia de Optuna
    if sampling_type == "RandomOverSampler":
        sampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
    elif sampling_type == "SMOTE":
        sampler = SMOTE(sampling_strategy='auto', random_state=42)
    elif sampling_type == "BorderlineSMOTE":
        sampler = BorderlineSMOTE(sampling_strategy='auto', random_state=42)
    elif sampling_type == "SVMSMOTE":
        sampler = SVMSMOTE(sampling_strategy='auto', random_state=42)
    else:
        raise ValueError("Tipo de muestreo no soportado")

    # Aplicar SMOTE al conjunto de entrenamiento actual
    X_resampled, y_resampled = sampler.fit_resample(X_train_scaled, y_train)

    # Initialize KNN and Decision Tree classifiers
    knn_model = KNeighborsClassifier(n_neighbors=knn_n_neighbors, weights=knn_weights)
    dt_model = DecisionTreeClassifier(max_depth=dt_max_depth, criterion=dt_criterion)
    
    # Stacking Classifier with KNN and Decision Tree as base estimators
    estimators = [('knn', knn_model), ('decision_tree', dt_model)]
    stacking_model = StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier())
    stacking_model.fit(X_resampled, y_resampled)
    
    y_pred = stacking_model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=400)

best_params = study.best_params
best_accuracy = study.best_value

print("Best Parameters Found:", best_params)
print("Corresponding Accuracy:", best_accuracy)


[I 2023-11-26 16:36:25,828] A new study created in memory with name: no-name-a04c6cf3-4925-48e1-ba8e-980de860d12d
[I 2023-11-26 16:36:25,938] Trial 0 finished with value: 0.639344262295082 and parameters: {'knn_n_neighbors': 47, 'knn_weights': 'distance', 'dt_max_depth': 2, 'dt_criterion': 'entropy', 'sampling_type': 'BorderlineSMOTE'}. Best is trial 0 with value: 0.639344262295082.
[I 2023-11-26 16:36:25,967] Trial 1 finished with value: 0.6475409836065574 and parameters: {'knn_n_neighbors': 14, 'knn_weights': 'distance', 'dt_max_depth': 4, 'dt_criterion': 'entropy', 'sampling_type': 'BorderlineSMOTE'}. Best is trial 1 with value: 0.6475409836065574.
[I 2023-11-26 16:36:26,000] Trial 2 finished with value: 0.5655737704918032 and parameters: {'knn_n_neighbors': 23, 'knn_weights': 'uniform', 'dt_max_depth': 5, 'dt_criterion': 'entropy', 'sampling_type': 'BorderlineSMOTE'}. Best is trial 1 with value: 0.6475409836065574.
[I 2023-11-26 16:36:26,033] Trial 3 finished with value: 0.57377049

Best Parameters Found: {'knn_n_neighbors': 1, 'knn_weights': 'uniform', 'dt_max_depth': 7, 'dt_criterion': 'entropy', 'sampling_type': 'BorderlineSMOTE'}
Corresponding Accuracy: 0.8032786885245902


In [29]:
X = data.drop('label', axis=1)
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Normalizing data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [30]:


def objective(trial):
    # KNN parameters
    knn_n_neighbors = trial.suggest_int('knn_n_neighbors', 1, 50)
    knn_weights = trial.suggest_categorical('knn_weights', ['uniform', 'distance'])
    
    # Decision Tree parameters
    dt_max_depth = trial.suggest_int('dt_max_depth', 1, 10)
    dt_criterion = trial.suggest_categorical('dt_criterion', ['gini', 'entropy'])
    
    # Final estimator (Decision Tree) parameters for Stacking Classifier
    final_dt_max_depth = trial.suggest_int('final_dt_max_depth', 1, 10)
    final_dt_criterion = trial.suggest_categorical('final_dt_criterion', ['gini', 'entropy'])

    # Initialize KNN and Decision Tree classifiers
    knn_model = KNeighborsClassifier(n_neighbors=knn_n_neighbors, weights=knn_weights)
    dt_model = DecisionTreeClassifier(max_depth=dt_max_depth, criterion=dt_criterion)
    final_dt_model = DecisionTreeClassifier(max_depth=final_dt_max_depth, criterion=final_dt_criterion)

    # Stacking Classifier with KNN and Decision Tree as base estimators
    estimators = [('knn', knn_model), ('decision_tree', dt_model)]
    stacking_model = StackingClassifier(estimators=estimators, final_estimator=final_dt_model)
    stacking_model.fit(X_train_scaled, y_train)
    
    y_pred = stacking_model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
best_accuracy = study.best_value

print("Best Parameters Found:", best_params)
print("Corresponding Accuracy:", best_accuracy)



[I 2023-11-26 16:36:44,935] A new study created in memory with name: no-name-d3c5d1f1-8397-42b1-b685-0ca40a2f4bd2
[I 2023-11-26 16:36:44,968] Trial 0 finished with value: 0.639344262295082 and parameters: {'knn_n_neighbors': 27, 'knn_weights': 'uniform', 'dt_max_depth': 2, 'dt_criterion': 'entropy', 'final_dt_max_depth': 6, 'final_dt_criterion': 'entropy'}. Best is trial 0 with value: 0.639344262295082.
[I 2023-11-26 16:36:45,002] Trial 1 finished with value: 0.639344262295082 and parameters: {'knn_n_neighbors': 36, 'knn_weights': 'distance', 'dt_max_depth': 6, 'dt_criterion': 'gini', 'final_dt_max_depth': 5, 'final_dt_criterion': 'entropy'}. Best is trial 0 with value: 0.639344262295082.
[I 2023-11-26 16:36:45,038] Trial 2 finished with value: 0.6967213114754098 and parameters: {'knn_n_neighbors': 26, 'knn_weights': 'distance', 'dt_max_depth': 7, 'dt_criterion': 'gini', 'final_dt_max_depth': 6, 'final_dt_criterion': 'entropy'}. Best is trial 2 with value: 0.6967213114754098.
[I 2023-1

Best Parameters Found: {'knn_n_neighbors': 3, 'knn_weights': 'distance', 'dt_max_depth': 6, 'dt_criterion': 'entropy', 'final_dt_max_depth': 4, 'final_dt_criterion': 'entropy'}
Corresponding Accuracy: 0.8032786885245902


In [31]:
# Entrenar un nuevo modelo con los mejores parámetros encontrados por Optuna
best_knn_n_neighbors = best_params['knn_n_neighbors']
best_knn_weights = best_params['knn_weights']
best_dt_max_depth = best_params['dt_max_depth']
best_dt_criterion = best_params['dt_criterion']
best_final_dt_max_depth = best_params['final_dt_max_depth']
best_final_dt_criterion = best_params['final_dt_criterion']

best_knn_model = KNeighborsClassifier(n_neighbors=best_knn_n_neighbors, weights=best_knn_weights)
best_dt_model = DecisionTreeClassifier(max_depth=best_dt_max_depth, criterion=best_dt_criterion)
best_final_dt_model = DecisionTreeClassifier(max_depth=best_final_dt_max_depth, criterion=best_final_dt_criterion)

best_estimators = [('knn', best_knn_model), ('decision_tree', best_dt_model)]
best_stacking_model = StackingClassifier(estimators=best_estimators, final_estimator=best_final_dt_model)
best_stacking_model.fit(X_train_scaled, y_train)



StackingClassifier(estimators=[('knn',
                                KNeighborsClassifier(n_neighbors=3,
                                                     weights='distance')),
                               ('decision_tree',
                                DecisionTreeClassifier(criterion='entropy',
                                                       max_depth=6))],
                   final_estimator=DecisionTreeClassifier(criterion='entropy',
                                                          max_depth=4))

In [32]:
from sklearn.metrics import classification_report

# Realizar predicciones en el conjunto de validación
y_pred_val = best_stacking_model.predict(X_test_scaled)

# Obtener el classification report
classification_rep = classification_report(y_test, y_pred_val)
print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.87      0.81        61
           1       0.85      0.72      0.78        61

    accuracy                           0.80       122
   macro avg       0.80      0.80      0.79       122
weighted avg       0.80      0.80      0.79       122



In [33]:
# Cargar y preparar los datos de validación
df_val = pd.read_csv('df_validar.csv')
df_val


,Unnamed: 0,username,grade,seq_4_avg,quiz_avg,seq_0_avg,seq_1_avg,seq_2_avg,seq_3_avg,duracion_sesion_avg,...,num_eventos_seq_1,num_eventos_seq_2,num_eventos_seq_3,num_eventos_seq_4,avg_time_between_sessions,std_time_between_sessions,nightactivity_2_to_5,nightactivity_0_to_2,sesionesde_2_5,sesionesde_0_2
0,0,05743498d061bf472df897f1e5bdda6d62e99312c952e3...,0.94,1.0,0.905172,1.0,0.977273,0.944444,0.666667,3.955454,...,175.0,92.0,22.0,0.0,3.250000,3.058637,97,197,4,1
1,1,afca888290f061f966a5f4ab433b53ac73734e00bdfce6...,0.79,1.0,0.758621,0.5,0.636364,0.555556,0.333333,9.797653,...,182.0,61.0,2.0,0.0,6.100000,5.989806,130,78,1,6
2,2,95ee01a5a7e3c41023fc379d6acb0f47860f2149caacb1...,0.85,-1.0,0.818966,1.0,0.795455,0.777778,0.333333,6.902268,...,241.0,66.0,23.0,3.0,4.538462,4.012800,210,215,1,2
3,3,4f824a96d26c45a3e047fd26f1f164371e86b4cab53895...,0.80,1.0,0.773103,1.0,0.795455,0.518889,0.666667,5.655644,...,156.0,96.0,16.0,4.0,3.800000,2.858571,251,125,6,4
4,4,ef14776d7e8bc0b6ad12b59d9b12e2d742e962dfc415b0...,0.95,1.0,0.913793,1.0,0.772727,1.000000,1.000000,4.238369,...,400.0,71.0,21.0,4.0,4.133333,3.997618,197,349,8,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,138,1d3bade754d542a25f8d826f484e9ef1c53eec484cd300...,0.78,1.0,0.750000,0.5,0.522727,0.666667,1.000000,3.749897,...,271.0,65.0,42.0,8.0,7.166667,5.600595,334,59,4,1
139,139,807e0f39ca606bc9c539abe9efce7c520a7193a29ec905...,0.95,1.0,0.913793,1.0,0.818182,0.944444,1.000000,4.642490,...,1420.0,912.0,51.0,9.0,3.615385,3.228479,254,494,4,3
140,140,c1d95ba9872db04aed5ab6c1ab1da219e20b98a971c444...,0.74,1.0,0.710000,0.5,0.522727,0.537778,1.000000,2.432727,...,707.0,359.0,54.0,2.0,3.357143,3.103525,838,299,8,4
141,141,65a4a3bd05c0e27b7bba2ee5b95f50c9af51ba46bcdd17...,0.59,1.0,0.568966,0.0,0.272727,0.166667,0.333333,6.289711,...,283.0,109.0,11.0,0.0,4.500000,6.603871,4,218,0,1


In [34]:
X_new = df_val.drop(['username', 'Unnamed: 0'], axis=1)
X_new

,grade,seq_4_avg,quiz_avg,seq_0_avg,seq_1_avg,seq_2_avg,seq_3_avg,duracion_sesion_avg,duracion_sesion_std,duracion_EOL,...,num_eventos_seq_1,num_eventos_seq_2,num_eventos_seq_3,num_eventos_seq_4,avg_time_between_sessions,std_time_between_sessions,nightactivity_2_to_5,nightactivity_0_to_2,sesionesde_2_5,sesionesde_0_2
0,0.94,1.0,0.905172,1.0,0.977273,0.944444,0.666667,3.955454,5.099847,65.0,...,175.0,92.0,22.0,0.0,3.250000,3.058637,97,197,4,1
1,0.79,1.0,0.758621,0.5,0.636364,0.555556,0.333333,9.797653,7.638587,61.0,...,182.0,61.0,2.0,0.0,6.100000,5.989806,130,78,1,6
2,0.85,-1.0,0.818966,1.0,0.795455,0.777778,0.333333,6.902268,6.676685,59.0,...,241.0,66.0,23.0,3.0,4.538462,4.012800,210,215,1,2
3,0.80,1.0,0.773103,1.0,0.795455,0.518889,0.666667,5.655644,7.179351,57.0,...,156.0,96.0,16.0,4.0,3.800000,2.858571,251,125,6,4
4,0.95,1.0,0.913793,1.0,0.772727,1.000000,1.000000,4.238369,4.568700,62.0,...,400.0,71.0,21.0,4.0,4.133333,3.997618,197,349,8,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,0.78,1.0,0.750000,0.5,0.522727,0.666667,1.000000,3.749897,2.704383,43.0,...,271.0,65.0,42.0,8.0,7.166667,5.600595,334,59,4,1
139,0.95,1.0,0.913793,1.0,0.818182,0.944444,1.000000,4.642490,3.980590,47.0,...,1420.0,912.0,51.0,9.0,3.615385,3.228479,254,494,4,3
140,0.74,1.0,0.710000,0.5,0.522727,0.537778,1.000000,2.432727,2.767814,47.0,...,707.0,359.0,54.0,2.0,3.357143,3.103525,838,299,8,4
141,0.59,1.0,0.568966,0.0,0.272727,0.166667,0.333333,6.289711,7.184712,45.0,...,283.0,109.0,11.0,0.0,4.500000,6.603871,4,218,0,1


In [35]:
# Ajustar el scaler a tus datos de entrenamiento y luego transformar los datos
X_val = scaler.transform(X_new)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- duracion_EOL
- duracion_sesion_avg
- duracion_sesion_std
- load_video_mean
- load_video_std
- ...


In [ ]:

# Realizar predicciones con el modelo cargado
y_pred = best_stacking_model.predict(X_val)


In [ ]:
# Agregar las predicciones como una nueva columna al conjunto de datos original
resultados = pd.DataFrame({'username': df_val['username'], 'Prediccion': y_pred})

# Guardar los resultados en un archivo CSV
resultados.to_csv('resultados_predichos1.csv', index=False)